In [1]:
include("../src/Utero.jl")
using .Utero 

In [2]:
ctx = ⬅Context()
ForwardBackward!(ctx, x -> 2.0*x + x, AddParams!(ctx, 7.0))

(21.0, Any[3.0])

In [3]:
# should be 12.8
ctx = ⬅Context()
ForwardBackward!(ctx, x -> x^2.0 + x^2.0, AddParams!(ctx, 3.2)) 

(20.480000000000004, Any[12.8])

In [4]:
# Gradient should be 56.21
ctx = Utero.⬅Context()
ForwardBackward!(ctx, w -> 2.0 * (((2.0*w + 12.0) - w) * w) + sin(w^2.0), AddParams!(ctx, 7.0))

(265.0462473472405, Any[56.20829561241092])

In [5]:
# should be approx. (27.53, 3.0428)
ctx = Utero.⬅Context()
x, y = AddParams!(ctx, 13.2, 89.0)
ForwardBackward!(ctx, (x, y) -> (cos(y)^3 * 4sin(x))^2 + cos(x*y)^2, x, y) 

(1.0748674472530375, Any[27.53002464314667, 3.0428042273213602])

In [6]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W[5:5]' * W[5:5], 3)

(0.009026837415936791, Any[sparse([5], [1], [0.19001934023605904], 30, 1)])

In [7]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W' * W, 3)
print(size(grads[1]))

(30, 1)

In [8]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W' * W, 3)
print(size(grads[1]))

(30, 1)

In [9]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(2,1))
(out, grads) = ForwardBackward!(ctx, _ -> W .* 2, nothing)
@show size(W.val)
print(size(grads[1]))
print(grads[1])


size(W.val) = (2, 1)
(2, 1)

[2.0; 2.0;;]

In [10]:
W = rand(2,1)
@show size(((W' * W) .* W)' * W)

U1, C1 = ⬅Dual(adjoint, W)
@show size(U1)
U2, C2 = ⬅Dual(*, U1, W)
@show size(U2)
U3, C3 = ⬅Dual(Base.broadcasted, *, U2, W)
@show size(U3)
U4, C4 = ⬅Dual(adjoint, U3)
@show size(U4)
U5, C5 = ⬅Dual(*, U4, W)
@show size(U5)

DU5 = [1]
@show size(DU5)
DU4, _ = C5(DU5)
@show size(DU4)
DU3 = C4(DU4)
@show size(DU3)
DU2, _ = C3(DU3)
@show size(DU2)
DU1, _ = C2(DU2)
@show size(DU1)




size(((W' * W) .* W)' * W) = (1, 1)
size(U1) = (1, 2)
size(U2) = (1, 1)
size(U3) = (2, 1)
size(U4) = (1, 2)
size(U5) = (1, 1)
size(DU5) = (1,)
size(DU4) = 

(1, 2)
size(DU3) = (2, 1)
size(DU2) = (1, 1)
size(DU1) = (1, 2)


(1, 2)

In [11]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(2,1))
(out, grads) = ForwardBackward!(ctx, _ -> ((W' * W) .* W)' * W, nothing)
@show size(W.val)
@show size(grads[1])

size(W.val) = (2, 1)
size(grads[1]) = (2, 1)


(2, 1)

In [16]:
Softmax(rand(2,1))
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(2,1))
@show W
(out, grads) = ForwardBackward!(ctx, _ -> Softmax(W), nothing)
@show grads[1]

W = Main.Utero.⬅Tracker{Matrix{Float64}}([0.46985385321909434; 0.7529370539979908;;], UUID("5e8d7e6e-c918-11ee-3e04-7b8bd7c6ccd8"), Any[], Any[])
grads[1] = [6.385585576818904; 8.47504868154015;;]


2×1 Matrix{Float64}:
 6.385585576818904
 8.47504868154015